###  installing and importing packages

In [6]:
pip install betterreads

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
from betterreads import client

###  Data collection approach

I am starting with a Kaggle dataset from  https://www.kaggle.com/zygmunt/goodbooks-10k

# TODO < include > eda


additional data collection for 
*Book description 
*number of pages
*is_book_enum

In [22]:
#  Get the API keys here https://www.goodreads.com/api/keys
gc = client.GoodreadsClient('zIHHGGHATvYay6dTTZ1AqA',
                            'xFiCr7PBabSezd7zpXhqNyvsIDFfoX4ftC6Pww7v2g')

In [23]:
books = pd.read_csv('../data/books.csv')
print( f"shape of the books dataframe is {books.shape}")
books.head(1)

shape of the books dataframe is (10000, 23)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


In [24]:
books.isna().sum()

book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

In [25]:
books[books['original_title'].isna()][['title','original_title']]

,title,original_title
74,"Bridget Jones's Diary (Bridget Jones, #1)",NaN
142,All the Light We Cannot See,NaN
209,"Vampire Academy (Vampire Academy, #1)",NaN
214,Ready Player One,NaN
256,Alice in Wonderland,NaN
...,...,...
9942,Sherlock Holmes and the Case of the Hound of t...,NaN
9944,"Rock Bottom (Tristan & Danika, #2)",NaN
9956,"Inside of a Dog: What Dogs See, Smell, and Know",NaN
9957,The Pacific,NaN


In [26]:
def get_book_id_list_for_missing_column(column):
    books[column].isna().sum()
    missing_desc_books = books[books[column].isna()]['book_id'].values
    return missing_desc_books

get_book_id_list_for_missing_column('description')

KeyError: 'description'

In [9]:
missing_isbn_list = get_book_id_list_for_missing_column('isbn')
missing_isbn_list

array([], dtype=int64)

In [12]:
# get  missing ISBN numbers
for i in missing_isbn_list:
    goodreads_book_id = books.loc[books['book_id'] == i,'goodreads_book_id'].values[0]
    try:
        book_obj= get_book(goodreads_book_id)
        books.loc[books['book_id'] == i,'isbn']= book_obj.isbn
    except:  
        books.loc[books['book_id'] == i,'isbn']= 0
len(books[books['isbn']==0])

700

In [27]:
#https://betterreads.readthedocs.io/en/latest/book.html
#for i in range(1,4000):
for i in missing_desc_books:
    goodreads_book_id = books.loc[books['book_id'] == i,'goodreads_book_id'].values[0]
    try:
        book_obj= get_book(goodreads_book_id)
        print( f"{i},{book_obj}")
        books.loc[books['book_id'] == i,'description']= book_obj.description
        try:
            books.loc[books['book_id'] == i,'num_pages']=  book_obj.num_pages
        except:
            books.loc[books['book_id'] == i,'num_pages']=  0

        books.loc[books['book_id'] == i,'is_ebook']=book_obj.is_ebook  
    except:
        books.loc[books['book_id'] == i,'num_pages']=  0
        books.loc[books['book_id'] == i,'description'] =""
        books.loc[books['book_id'] == i,'is_ebook'] =False
        print(f"failed to parse {i}")

728,Brown Bear, Brown Bear, What Do You See?
977,Inferno
992,The Twilight Saga (Twilight, #1-4)
994,All Creatures Great and Small (All Creatures Great and Small, #1)
1108,The Complete Fairy Tales
1113,Thus Spoke Zarathustra
1329,Howl and Other Poems
1338,The Book of Mormon: Another Testament of Jesus Christ
1382,Anne Rice's The Vampire Lestat: A Graphic Novel
1432,The Communist Manifesto
failed to parse 1475
1666,Shakespeare's Sonnets
1679,Leaves of Grass
1691,Narrative of the Life of Frederick Douglass
1754,The Mortal Instruments Boxed Set: City of Bones; City of Ashes; City of Glass (The Mortal Instruments, #1-3)
1935,Kindle Paperwhite User's Guide
2068,Out of the Dust
2100,Batman: The Dark Knight Returns #1
failed to parse 2139
failed to parse 2151
2397,The Further Adventures of Sherlock Holmes: After Sir Arthur Conan Doyle (Classic Crime)
2680,The Autobiography of Benjamin Franklin
2685,Killer (Pretty Little Liars, #6)
3054,Harry Potter and the Chamber of Secrets: Sheet Music for F

In [15]:
# Read book ratings
ratings = pd.read_csv("../data/ratings.csv")
print( f"shape of the books dataframe is {ratings.shape}")
ratings.head(1)

shape of the books dataframe is (5976479, 3)


,user_id,book_id,rating
0,1,258,5


In [16]:
books.rename({'original_title':'title'},inplace=True)
#books =pd.merge(books,ratings, how ='inner',left_on ='book_id',right_on='book_id')
#books.head(1)

In [17]:
book_tags = pd.read_csv("../data/book_tags.csv")
print( f"shape of the books dataframe is {book_tags.shape}")
book_tags.head(1)

shape of the books dataframe is (999912, 3)


,goodreads_book_id,tag_id,count
0,1,30574,167697


In [18]:
tags = pd.read_csv("../data/tags.csv")
print( f"shape of the books dataframe is {tags.shape}")
tags.head(1)

shape of the books dataframe is (34252, 2)


,tag_id,tag_name
0,0,-


In [19]:
book_tags =pd.merge(left =book_tags,right=tags, how="left")
book_tags.head(1)

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read


In [20]:
#books =pd.merge(books,book_tags, how ='inner',left_on ='goodreads_book_id',right_on='goodreads_book_id')
#books.head(1)

In [21]:
def get_book(id):
    book = gc.book(id)
    return book

In [22]:
books.to_csv("../data/books_desc.csv", index=False)

In [ ]:
# Combine all data frames in to one

######  references
https://pypi.org/project/Goodreads/

https://betterreads.readthedocs.io/en/latest/

https://www.kaggle.com/zygmunt/goodbooks-10k